In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:

from __future__ import print_function
import pandas as pd
import datetime as dt
from sklearn import metrics
import tensorflow as tf
import numpy as np
import math
from tensorflow.python.data import Dataset
from sklearn.model_selection import train_test_split
import tensorflow.losses as loss_utils
from sklearn.preprocessing import LabelEncoder
import math
import os


In [ ]:

data_set = pd.read_csv('/content/drive/My Drive/Ass4_data/train.csv', sep = ',', nrows = 100000);
data_set.fillna(method='bfill',inplace=True)

In [ ]:
time = pd.to_datetime(data_set['pickup_datetime'], format="%Y-%m-%d %H:%M:%S UTC", exact = True)
data_set['month'] = [i.month for i in time]
data_set['date'] = [i.day for i in time]
data_set['hour'] = [i.hour for i in time]
data_set['minute'] = [i.minute for i in time]



In [ ]:
#fitering corrupt/0 entry fields (some have  0 latitude/longitude)
#normalise 
input = data_set.to_numpy()
input = input[np.all(input != 0, axis=1)]
input[:, 3] += 73
input[:, 4] -= 40
input[:, 5] += 73
input[:, 6] -= 40

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input[:, 3:],input[:, 1],shuffle=True,test_size=0.1)

x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')

#print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

In [ ]:
if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please select TPU in runtime!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address: ', tpu_address)

TPU address:  grpc://10.116.207.90:8470


In [ ]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, 0.01, 0.01))

def init_biases(shape):
    return tf.Variable(tf.zeros(shape))

def fc_layer(inputs, input_shape, output_shape, activation=tf.nn.relu):

    weights = init_weights([input_shape, output_shape])
    biases = init_biases([output_shape])
    
    layer = tf.matmul(inputs, weights) + biases
    
    if activation != None:
        layer = activation(layer)
        
    return layer

In [ ]:
inputs =  tf.placeholder(tf.float32, [None, 9], name='Inputs')
targets =  tf.placeholder(tf.float32, [None, 1], name='Targets')


In [ ]:
#defining the network
l1 = fc_layer(inputs, 9, 10)
l2 = fc_layer(l1, 10, 10)
l3 = fc_layer(l2, 10, 10)
l4 = fc_layer(l3, 10, 1, activation=None)
predictions = l4

cost = loss2 = tf.reduce_mean(tf.squared_difference(targets, predictions))
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

epochs = 1000
batch_size = 1000

In [ ]:
from tqdm import tqdm

with tf.Session(tpu_address) as sess:
    sess.run(tf.global_variables_initializer())
    #TRAINING PORTION OF THE SESSION
    for i in tqdm(range(epochs)):
        idx = np.random.choice(len(x_train), batch_size, replace=True)
        x_batch = x_train[idx, :]
        y_batch = y_train[idx]
        y_batch = np.reshape(y_batch, (len(y_batch), 1))
        
        batch_loss, opt = sess.run([cost, optimizer], feed_dict={inputs:x_batch, targets:y_batch})
        
        if i % 100 == 0:
            print(" error: " , batch_loss)
    
    #TESTING PORTION OF THE SESSION
    preds = sess.run([predictions], feed_dict={inputs:x_test})
    preds = np.reshape(preds, (len(y_test)))
    print(max(preds), min(preds))


  1%|          | 6/1000 [00:00<01:30, 10.95it/s]

 error:  207.12029


 15%|█▌        | 152/1000 [00:00<00:12, 66.61it/s]

 error:  115.423546


 29%|██▉       | 288/1000 [00:01<00:04, 150.79it/s]

...

 error:  103.73591


 99%|█████████▉| 990/1000 [00:02<00:00, 457.42it/s]

 error:  113.1389


100%|██████████| 1000/1000 [00:02<00:00, 368.72it/s]


487.757 1.700698


In [ ]:
rmse = 0
for i in range(len(preds)):
    rmse += (preds[i] - y_test[i])**2
rmse /= len(preds)
print(rmse**0.5)

10.845746120690487


In [ ]:
preds


array([11.006992, 10.022571, 11.041456, ...,  8.790719, 11.237871,
       11.929437], dtype=float32)